## Group Assignment
### Team Number: 05
### Team Member Names: Piero Camposeo, Sathun Suthakaran, Ishaan Bansal
### Team Strategy Chosen: SAFE

## Overview:

We want to generate an unusually "safe" portfolio. Rather than maximize returns, we want to keep returns as close as possible to zero. The goal is to generate a portfolio, based on a random set of tickers, whose projected value over an arbitrary number of days is as close to its starting value as possible. Specifically, our code is made for a 5-day run-time of the portfolio, but the model used is viable for any period of time (though it is often not preferred).

The code is split into three sections:

1. DATA CLEANING
2. CHOOSING THE STOCKS
3. CREATING THE PORTFOLIO

Data cleaning involves removing tickers that do not fit a certain criteria, chosen by the user.

Choosing the stocks involves picking the least volatile tickers, which we will utilize in our final portfolio.

Creating the portfolio involves generating randomly weighted portfolios, comprised of our tickers. The final portfolio will be that which has the expected return closest to 0. We will calculate expected return based on a <b>modified, Multi-Security Capital Asset Pricing Model</b> (below).

![Modified, Multi-Security CAPM](Pricing-Model-Image.png)

The code should output the final .......****************

In [33]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
import threading
import time
from queue import Queue

In [34]:
#Read in csv file of tickers
csv_file = "Tickers.csv"

# Make values into a list
tickers = pd.read_csv(csv_file)
ticker_list = tickers.iloc[:, 0].values.tolist()
ticker_list.insert(0, tickers.columns[0])

In [35]:
#Consumes desired amount of trading days within a month, a dataframe of days, a dstaframe of months, and a list of months that cooresponds to the months
#    on the dataframe, outputs the monthly dataframe with only months that have 'daycount' amount of trading days for that stock
#Note: Function will cover up to the last date. For example, the function will not produce October for a dataframe which ends on October 1st.
#   In our code, we simply passed a days dataframe that ends one month after the months dataframe does (*)
def df_with_valid_months(daycount, df_days, df_months, months):
    for month in range (len(months)):
        if len(df_days.filter(like = months[month], axis = 0)) < daycount:
            df_months.drop(df_months.index[month], axis = 0, inplace = True) #Drop any months with not enough trading days
            
    return(df_months)

#Consumes historical data for a stock, outputs average volume
def average_volume(df_hist):
    volume = df_hist['Volume'].dropna().values.tolist()
    vol_av = np.mean(volume)

    return(vol_av)


## 1. DATA CLEANING: 

This cell is where the data cleaning happens.

THE GOAL:

•	Ignore any tickers that do not reference a valid stock denominated in USD, i.e., only use US listed stocks.

•	Use only tickers that reference stocks which have an average monthly volume of at least 200,000 shares, as calculated based on the time interval of January 01, 2022 to October 31, 2022. A month is defined as a calendar month. We will drop any month that does not have at least 20 trading days.

THE PROCESS:

Firstly, we have chosen to implement multithreading, in order to speed up the process. The data cleaning is the most time-consuming code in the program to run. Due to this, we have two main functions (currency_check and volume_check), running on two seperate threads, c and v. These functions are defined recursively; If too many tickers are passed into the function (i.e a list of tickers that is longer than the 'max_length_lst' variable in each function), the function calls on a number of threads to run the function with smaller, sublists. When a small enough quantity of tickers is passed to the function, it will perform the criteria check for the tickers. Valid tickers get passed to the function's respective queue. Finally, the queue is cleared, and tickers that fit both criterion are outputted and saved as our tickers.

Note 1: Currency check generates more threads per application than volume_check because it takes significantly longer to run. This is also why its 'max_list_length' variable is smaller

Note 2: For the sake of comparison, when tested by our team:

- Unthreaded, non-recursive code takes ~13 minutes to run
- Multithreaded, non-recursive code takes ~5-10 minutes to run
- <b>Multithreaded, recursive code takes ~1 minute to run</b>

Note 3: 'max_length_lst' may need to be updates for larger lists of tickers

Our code checks each of the two criterion mentioned in the goal (one for each function), and helps to generate two lists. Each list contains tickers which reference stocks that have satisfied the criteria of the functions. As we want stocks that fit both criterion, our final list of (valid) tickers is the set intersection of the two lists which each function produces.


In [36]:
#dates for volume calculation, end date for valid months function (see (*))
vol_date_s = '2022-01-01'
vol_date_e = '2022-10-02'
endDForValidMonthsFunct = str((pd.to_datetime(vol_date_e) + relativedelta(months = 1)).strftime("%Y-%m-%d"))

#for df_with_valid_months function, creates a series of months from vol_date_s to vol_date_e
months = pd.date_range(vol_date_s, vol_date_e, freq='MS').strftime("%Y-%m").tolist()

#As outlined in assignment information
minimumMonthlyTradingDays = 20
minimum_volume = 200000
required_denomination = 'USD'

#Here we will put valid stocks that fit our criteria
tickers = []

currency_valid = []
volume_valid = []

#Queue variables to store tickers
queueC = Queue()
queueV = Queue()

#Consumes a list of tickers, used to check if they are denominated in 'required_denomination'.
def currency_check(ticker_list):
    valid = []
    max_length_lst = 5

    #When there are an appropriate number of tickers in ticker_list, the function performs currency_check
    if len(ticker_list) <= max_length_lst:

        #Retrieve financial currency from each ticker. If it is 'required_denomination', it is a valid ticker. Else, we don't care so we set it as nan
        for ticker in ticker_list:
            tick = yf.Ticker(ticker)
            try:
                currency = tick.info['financialCurrency']
            except:
                currency = 'nan'
        
            #Check criteria, add to list of valid tickers for currency
            if currency == required_denomination:
                valid.append(ticker)

        #Pass our valid tickers to the QueueC (Queue Currency), so that they may be called on later
        queueC.put(valid)

    #If there are too many tickers in ticker_lst, we run a recursive application of currency_check with five threads, with the arguments being one of
    #Five equal parts of ticker_list  
    else:
        ticker_split = np.array_split(ticker_list, 5)

        cSub1 = threading.Thread(target = currency_check, args = (ticker_split[0],))
        cSub2 = threading.Thread(target = currency_check, args = (ticker_split[1],))
        cSub3 = threading.Thread(target = currency_check, args = (ticker_split[2],))
        cSub4 = threading.Thread(target = currency_check, args = (ticker_split[3],))
        cSub5 = threading.Thread(target = currency_check, args = (ticker_split[4],))

        cSub1.start()
        cSub2.start()
        cSub3.start()
        cSub4.start()
        cSub5.start()

        cSub1.join()
        cSub2.join()
        cSub3.join()
        cSub4.join()
        cSub5.join()
 
#Consumes a list of tickers, used to check if they have an average monthly volume of 'minimum_volume' or more over each month with
#'minimumMonthlyTradingDays' or more trading days
def volume_check(ticker_list):
    valid = []
    max_length_lst = 20
    
    #When there are an appropriate number of tickers in ticker_list, the function performs currency_check
    if len(ticker_list) <= max_length_lst:

        #Retrieve mnthly average volume from each ticker, if it is greater than or equal to minimum_volume, it is a valid ticker. Else, we ignore it.
        for ticker in ticker_list:
            tick = yf.Ticker(ticker)
        
            try:
                #Check for Delisted ticker + Creating histories. If 'check' returns an error, the code continues. Without this try/except,
                #Error code for delisted stock appears twice
                tick_hist = tick.history(start = vol_date_s, end  = vol_date_e, interval = '1mo').dropna()
                check = tick_hist.iloc[0,0]
                #Custom dataframe for valid months check (daily, ending one month after monthly dataframe)
                tickHistForValidMonthsCheck = tick.history(start = vol_date_s, end  = endDForValidMonthsFunct, interval = '1d').dropna()
 
                #Dataframe with valid months only
                tickHistWithValidMonths = df_with_valid_months(minimumMonthlyTradingDays, tickHistForValidMonthsCheck, tick_hist, months)

                #Call on average_volume function to compute and average volume
                monthly_average_volume = 0 #Initialize this to 0 to prevent errors
                monthly_average_volume = average_volume(tickHistWithValidMonths)
        
                #Check criteria, add to list of valid tickers for volume
                if monthly_average_volume >= minimum_volume:
                    valid.append(ticker)
       
            except:
                pass

        #Pass our valid tickers to the QueueV (Queue Volume), so that they may be called on later
        queueV.put(valid)

    #If there are too many tickers in ticker_lst, we run a recursive application of volume_check with two threads, with the arguments being one of
    #two equal parts of ticker_list    
    else:
        ticker_split = np.array_split(ticker_list, 2)

        vSub1 = threading.Thread(target = volume_check, args = (ticker_split[0],))
        vSub2 = threading.Thread(target = volume_check, args = (ticker_split[1],))

        vSub1.start()
        vSub2.start()

        vSub1.join()
        vSub2.join()
        
#Creating threads for currency_check and volume_check function
c = threading.Thread(target = currency_check, args = (ticker_list,))
v = threading.Thread(target = volume_check, args = (ticker_list,))

#Start thread executions
c.start()
v.start()

#Wait for v to finish, grab all elements from queue
v.join()
while not queueV.empty():
    volume_valid.append(queueV.get())

#list of lists -> list of strings
volume_valid = [item for sublist in volume_valid for item in sublist]


#Wait for c to finish, grab all elements from queue
c.join()
while not queueC.empty():
    currency_valid.append(queueC.get())

#list of lists -> list of strings
currency_valid = [item for sublist in currency_valid for item in sublist]

#Valid tickers are the intersection of the two lists (Must have particular denomination and average monthly volume as outlined in assignment)
tickers = sorted([value for value in currency_valid if value in volume_valid])

print(f"\nValid Tickers: {tickers}")

- RTN: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted
- AGN: No data found, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted

Valid Tickers: ['AAPL', 'ABBV', 'ABT', 'ACN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CSCO', 'CVS', 'GM', 'GOOG', 'JPM', 'KMI', 'KO', 'LLY', 'LMT', 'MO', 'MON', 'MRK', 'MS', 'MSFT', 'NEE', 'NKE', 'ORCL', 'OXY', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'SBUX', 'SLB', 'SO', 'SPG', 'T', 'TGT', 'TXN', 'UNH', 'UNP', 'UPS', 'USB']


## 2. CHOOSING THE STOCKS:

The following cells are where we choose the stocks to be used for the final portfolio.

THE GOAL:
Since our overall goal is to have a portfolio which finishes as close to its starting value as possible, we naturally want a diversified and stable selection of stocks. With this in mind, we decided to choose as many stocks as allowed (so as to avoid unsystematic risk), with stocks chosen being the least volatile out of the available selection. Essentially, we want:

•	(if there are less than 25 tickers to choose from) every (valid) ticker available <b>OR</b> 25 tickers (maximum allowed as outlined in assignment information) comprised of:

•	The least volatile stocks available

    •	Note: Volatility, as we define it in this section, is represented by the standard deviation of daily returns (over a recent 5 month period, specifically)

THE PROCESS:

.......

Sathun Explanation box

In [37]:
#For closing dict. Should be updated when code is used in the future.
start = '2022-06-01'
end = '2022-11-01'

#Creates a DataFrame of closing prices from start_date to end_date for ticker_list
def get_closing_df(ticker_list, start_date, end_date):
    #dictionary of closing prices
    closing_dict = {}

    interval = '1d'

    #Fills closing_dict dictionary with closing prices for each ticker in closing_tickers
    for ticker in ticker_list:
        tick = yf.Ticker(ticker)
        tick_hist = tick.history(start = start_date, end = end_date, interval = interval)

        closing_dict[ticker] = tick_hist['Close']

    closing_df = pd.DataFrame(closing_dict)

    return(closing_df) 

#Creates a DataFrame of percent returns from start_date to end_date for a DataFrame of closing prices
def get_returns_from_close_df(closing_df):
    #Dictionary of returns
    returns = {}

    #Fills returns dictionary with the pct returns for each ticker in closing_df
    for price in closing_df:
        returns[price] = closing_df[price].pct_change()

    returns_df = pd.DataFrame(returns)

    return(returns_df)

#We will use these for analysis
tickers_closeDF = get_closing_df(tickers, start, end) 
tickers_returnDF = get_returns_from_close_df(tickers_closeDF)

#print("Closing Prices Preview:")
#tickers_closeDF.head()

print("Percent Returns Preview:")
tickers_returnDF.head()

Percent Returns Preview:


,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,SLB,SO,SPG,T,TGT,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2022-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-02,0.016811,0.004999,0.021116,0.026687,0.005705,0.031450,0.020391,0.075392,0.000818,-0.007450,...,-0.008141,0.005294,0.010345,-0.001414,0.028048,0.013187,-0.000589,0.018390,0.021306,0.008309
2022-06-03,-0.038556,0.002862,-0.007999,-0.005446,-0.015128,-0.025185,-0.013657,-0.008897,-0.013897,0.010629,...,0.025270,-0.009085,-0.021100,-0.013686,0.000808,-0.024842,-0.013509,-0.006167,0.007714,-0.004599
2022-06-06,0.005228,-0.004349,-0.001287,-0.000721,0.029674,0.019943,0.001139,-0.000072,-0.001105,-0.017496,...,-0.005266,0.001196,-0.003001,0.001914,-0.008507,-0.026519,0.009411,0.005223,0.008734,0.005968
2022-06-07,0.017586,0.017812,0.003951,0.005907,0.007120,-0.014344,0.009999,0.011347,0.005532,0.020141,...,0.036213,0.004512,0.024266,0.009551,-0.023110,0.012458,0.014117,0.015899,-0.008551,0.005933


Ishaan Explanation box

In [38]:
max_portfolios = 25

# Consumes a Dataframe of stocks and their respective daily % returns and produces a list of the 'max_portfolios' least volatile stocks, or if there's
# Less than 'max_portfolios' stocks to choose from, produces a list of all of the stocks. Also prints dataframe of final stocks and their respected
# volatility, as well as the number of stocks in the final portfolio
def lowest_volatile_stocks(pr_df):
    
    all_stock_vol = []
    numstocks_final = 0 #This stays 0 until changed by function

    # Performs the calculations for every column
    for pctchange in range (len(pr_df.columns)):
        # Calculates the standard deviation of the stock's daily percent return
        stock_vol = pr_df.iloc[:,pctchange].std()
        
        all_stock_vol.append(stock_vol)

    # Creates a dataframe that will store each stock and their volatility, sorts by non-decreasing volatility
    all_stock_vol_df = pd.DataFrame({"Stocks": pr_df.columns, "Daily % Returns Std": all_stock_vol,})
    all_stock_vol_df.sort_values(by = "Daily % Returns Std", inplace = True)
    all_stock_vol_df.reset_index(inplace = True, drop = True)

    print("Preview of Final Selection of Stocks Daily Returns:\n")

    #Creates a list of the 'max_portfolios' least volatile stocks in the dataframe if there are more than 'max_portfolios', 
    #else creates a list of all of the stocks. Prints dataframe
    if len(all_stock_vol_df) > max_portfolios:
        final_portfolio_lst = all_stock_vol_df['Stocks'].loc[0 : max_portfolios + 1].values.tolist()
        numstocks_final = max_portfolios

        print(all_stock_vol_df.iloc[0 : max_portfolios + 1].head())
    else:
        final_portfolio_lst = all_stock_vol_df['Stocks'].values.tolist()
        numstocks_final = len(all_stock_vol_df)

        print(all_stock_vol_df.head())

    print(f"\nStocks in Portfolio ({numstocks_final}): \n{final_portfolio_lst}")
    
    return(final_portfolio_lst)

ticklist_final = sorted(lowest_volatile_stocks(tickers_returnDF))
tickersReturn_final = tickers_returnDF[ticklist_final]

Preview of Final Selection of Stocks Daily Returns:

  Stocks  Daily % Returns Std
0    MON             0.001079
1     KO             0.012059
2    PEP             0.012235
3     CL             0.012836
4    MRK             0.013238

Stocks in Portfolio (25): 
['MON', 'KO', 'PEP', 'CL', 'MRK', 'PG', 'BMY', 'UNH', 'PM', 'CSCO', 'ABBV', 'SO', 'PFE', 'UPS', 'UNP', 'ABT', 'LMT', 'CVS', 'LLY', 'T', 'MO', 'COST', 'ACN', 'USB', 'KMI', 'MS', 'JPM']


Explanation box for beta

In [110]:
#Ishaan
#For beta Calculations

#For gathering data for the NYA stock within our designated time period
start = '2022-06-01'
end = '2022-11-01'

#Consumes a DataFrame of the percent returns of stocks and a stock to represent the "market". It then returns the beta of each stock in the DataFrame
#compared to the "market" inputted. The "market" stock must be a string of how the ticker appears in yfinance.
def portfolio_beta(pr_df, market_stock):

    # Creates a dataframe of the percent returns of the stock representing the market
    market = yf.Ticker(market_stock)
    market_close = market.history(start=start, end = end, interval = "1d")['Close']
    market_pr = market_close.pct_change()
    market_pr_df = pd.DataFrame({market_stock: market_pr})
    
    # Merges pr_df and the market_pr_df together and drops the first row which is NaN values because there is no percent return for the first
    #data point
    beta_data = pr_df.merge(market_pr_df, left_index = True, right_index= True)
    beta_data.drop(index=beta_data.index[0], inplace = True)

    #Calculates the variance of the stock representing the market
    MarketVar = market_pr_df[market_stock].var()
    #Creates the beta matrix of the merged DataFrame
    Beta_matrix = beta_data.cov()/MarketVar

    #Prints Beta Matrix
    print("Portfolio's Beta Matrix Preview:")
    print(Beta_matrix.head())

    #DataFrame of the Betas of each stock
    portfolio_stock_beta = pd.DataFrame({"Beta": Beta_matrix.iloc[:, (len(Beta_matrix) - 2)],})
    
    return portfolio_stock_beta

portfolio_betas = portfolio_beta(tickers_returnDF, "^NYA")

#Prints the Beta of each stock
print("Portfolio's Stocks' Betas Preview:")
portfolio_betas.head()


Portfolio's Beta Matrix Preview:
          AAPL      ABBV       ABT       ACN       AIG      AMZN       AXP  \
AAPL  2.483128  0.466178  1.133902  1.627584  1.499192  2.263525  1.665457   
ABBV  0.466178  1.281468  0.732077  0.572243  0.577564  0.598701  0.544243   
ABT   1.133902  0.732077  1.489534  1.139251  1.094858  1.279799  1.212666   
ACN   1.627584  0.572243  1.139251  1.769214  1.526428  1.980083  1.633033   
AIG   1.499192  0.577564  1.094858  1.526428  2.424994  1.697400  1.869641   

            BA       BAC      BIIB  ...        SO       SPG         T  \
AAPL  1.952248  1.356990  1.037191  ...  0.990636  1.439626  1.015706   
ABBV  0.496851  0.719956  1.016080  ...  0.548422  0.565767  0.542119   
ABT   1.143330  1.167813  1.177727  ...  0.761948  0.978356  0.606986   
ACN   1.791350  1.391634  1.549691  ...  0.868209  1.427098  0.877425   
AIG   1.746534  1.740551  1.166999  ...  0.743586  1.571155  0.910700   

           TGT       TXN       UNH       UNP       UPS     

,Beta
AAPL,1.107754
ABBV,0.642761
ABT,1.077014
ACN,1.171925
AIG,1.621465


Piero Intro to choosing portfolios

In [ ]:
#Piero Choose portfolios

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Piero Camposeo, Sathun Suthakaran, Ishaan Bansal

### Conclusion

### Sources
- 